In [6]:
import pandas as pd
from string import punctuation
import nltk
from nltk.corpus import stopwords
from datetime import datetime

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/marcus/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Load dataset

In [7]:
df = pd.read_pickle('data/extracted.pickle')


### Process Natural Language


In [40]:
stop_words = set(stopwords.words('english'))

def preprocess_docstrings(docstring):
    docstring = docstring.lower()
    docstring = docstring.translate(str.maketrans(' ', ' ', punctuation))
    docstring = ' '.join([w for w in docstring.split() if w.lower() not in stop_words])

    return docstring

In [10]:
# df.docstring = df.docstring.apply(preprocess_docstrings)
df.docstring.head()

0    Attempt to convert the specified value to epoc...
1    Generate a server side cookie given the cookie...
2    Generate httponly cookie from HS2 cookie\n@par...
3    Copies all files from source to target and set...
4            Saves the configuration info to the disk.
Name: docstring, dtype: object

### Process Code Strings

### Save preprocessed dataset

In [9]:
df.to_pickle('data/processed_' + str(datetime.now()) + '.pickle')